# Math. Methoden der Data Science
__Prof. Dr. Fabian Transchel__
### Labor 6: DBSCAN et al.

<hr />

__Aufgabe 1__: Wenden Sie DBSCAN auf den Datensatz telematik_agg_v2.csv an:

- Berechnen Sie elbow, silhouette, gap statistik.
- Wie viele Cluster scheinen plausibel?
- Wie sind die Cluster im Vergleich zu `driving_skill` angeordnet?
- Welche Bild ergibt sich für ein CLustering mit/ohne dieses Merkmal?
- Plotten Sie eine Auswahl von 3D-Schnitten durch den Merkmalsraum mit geeignetem Color-Coding.

In [12]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
telematik_db = pd.read_csv("telematik_agg_v2.csv", index_col=0)
print(len(telematik_db))

135603


In [11]:
cleaned_db = telematik_db.drop(["Unnamed: 0.1", "driving_skill"], axis = 1)
cleaned_db.head()

,DrivAge,rel_risk,Mileage,max_accel_x,min_accel_x,max_accel_y,min_accel_y,max_speed_mw,max_speed_urban,max_speed_rural,ratio_mw,ratio_urban,ratio_rural,ratio_night,ratio_precip,ratio_rush,four_accel_1,four_accel_2,four_accel_3,four_accel_4
0,52,0.025511,6235.337875,4.270571,-3.186554,2.236002,-1.696633,139.580307,131.528249,55.806158,0.147900,0.234289,0.617811,0.159878,0.182537,0.065222,2.470029,0.000814,82.928661,3782.818447
1,38,0.093298,2520.900336,1.787043,-3.118378,2.129723,-1.241005,157.229086,136.481766,58.827611,0.158301,0.348099,0.493600,0.162678,0.153850,0.138712,-3.454407,0.404569,30.738335,496.235516
2,44,0.119460,14136.719210,4.403333,-3.074536,2.512433,-1.387016,166.693425,136.615064,61.657666,0.133120,0.226000,0.640880,0.161343,0.200200,0.180049,-25.788232,3.060334,157.741334,3917.657266
3,32,0.023914,7718.314400,0.481826,-3.012558,1.783253,-2.248576,136.874746,131.570166,56.072814,0.155843,0.225024,0.619133,0.151244,0.165610,0.365876,2.066236,79.250061,174.778169,2958.453923
4,73,0.108091,3495.908846,14.967886,-2.792939,2.182035,-1.979150,164.069706,135.150505,60.160611,0.154932,0.196956,0.648112,0.156745,0.114058,0.000403,10.282886,0.000067,28.245054,792.581652


In [13]:
scaler = StandardScaler()
scaler.fit(cleaned_db)
transformed_scale = scaler.transform(cleaned_db)

scaled_df = pd.DataFrame(transformed_scale, columns = cleaned_db.columns)
scaled_df.head()

,DrivAge,rel_risk,Mileage,max_accel_x,min_accel_x,max_accel_y,min_accel_y,max_speed_mw,max_speed_urban,max_speed_rural,ratio_mw,ratio_urban,ratio_rural,ratio_night,ratio_precip,ratio_rush,four_accel_1,four_accel_2,four_accel_3,four_accel_4
0,0.457193,-0.082401,-0.213307,-1.182968,-1.596150,0.554772,0.407100,0.232225,-0.082785,-0.318348,-0.062355,-0.344322,0.350185,0.610592,1.303095,-1.318276,-0.148633,-0.447558,-0.186228,1.536733
1,-0.532747,2.127287,-1.044546,-2.157052,-0.939181,0.158571,1.421431,2.265933,2.570087,1.794334,0.895417,1.931710,-2.080549,0.834114,0.156915,-0.806850,-0.855284,-0.438501,-0.705507,-0.934745
2,-0.108487,2.980089,1.554913,-1.130896,-0.516698,1.585289,1.096379,3.356530,2.641475,3.773186,-1.423368,-0.510087,0.801631,0.727520,2.008869,-0.519176,-3.519203,-0.378922,0.558136,1.638130
3,-0.957007,-0.134481,0.118563,-2.668981,0.080539,-1.133047,-0.821648,-0.079543,-0.060336,-0.131895,0.669066,-0.529614,0.376062,-0.078652,0.626795,0.774021,-0.196797,1.330289,0.727648,0.916819
4,1.942104,2.609494,-0.826353,3.012710,2.196870,0.353590,-0.221846,3.054193,1.857126,2.726404,0.585189,-1.090929,0.943157,0.360482,-1.433015,-1.769356,0.783264,-0.447575,-0.730314,-0.711896


In [27]:
sampled_df = scaled_df.sample(frac = 0.1, replace=True, random_state=1)
print(len(sampled_df))
sampled_df.head()

13560


,DrivAge,rel_risk,Mileage,max_accel_x,min_accel_x,max_accel_y,min_accel_y,max_speed_mw,max_speed_urban,max_speed_rural,ratio_mw,ratio_urban,ratio_rural,ratio_night,ratio_precip,ratio_rush,four_accel_1,four_accel_2,four_accel_3,four_accel_4
128037,-0.391327,-0.859271,-0.203309,0.449583,0.673078,-2.306798,-0.034325,-0.825171,-0.839807,-0.832701,-1.101117,-0.292360,0.520092,-0.054779,-0.733958,-0.467218,-0.467257,-0.417697,2.533143,0.220722
5192,-0.532747,0.038960,-0.142555,-2.001458,-0.813464,-0.572093,0.123863,0.379712,0.006718,0.412354,-0.525996,-0.758431,0.853939,-0.971097,-0.747877,0.315920,1.013742,-0.390442,-0.468355,-0.318282
50057,0.245063,0.261815,-0.043716,-1.558623,0.169447,0.395382,1.006543,0.060230,0.159703,0.306626,-1.479986,0.211073,0.107978,-1.490014,0.213468,1.537747,-1.290325,-0.420398,1.016930,1.700051
109259,1.164293,-0.838214,-1.485689,-0.291506,0.446518,-1.080085,1.488635,-0.806638,-0.836452,-0.801974,1.102381,-0.619535,0.371966,-0.803877,0.063601,0.108829,0.821542,0.397934,-0.749867,-1.156111
73349,-0.391327,1.718700,2.193680,-0.867945,-1.728613,0.015579,0.524174,2.376325,1.034867,1.532222,0.079847,1.169911,-1.161775,1.333619,1.177539,-0.535549,-1.445530,-0.447567,0.055468,-0.174239


In [42]:
clustering = DBSCAN(eps = 2, min_samples = 5).fit(sampled_df)
labels = clustering.labels_
labels

array([-1, -1, -1, ..., -1, -1, -1])

In [43]:
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(n_clusters)
print(n_noise)

33
11875


In [44]:
EPS = np.linspace(0, 5, endpoint = False, retstep = 0.1, dtype = float)
elbow_func = lambda x: 

for epsilon in EPS:
    clustering = DBSCAN(eps = epsilon, min_samples = 5).fit(sampled_df)
    labels = clustering.labels_
    

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
        1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
        2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
        3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]),
 0.1)

__Aufgabe 2__: Laden Sie den neuen Datensatz cluster2.csv

- Bestimmen Sie 2...8 Cluster. Diskussion: Wie könnte man ohne Zielgröße den Erfolg messen?
- Welche Probleme stellen sich mit diesem Datensatz?
- Kann man durch geeignete Visualisierung etwas lernen?